In [79]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [80]:
#@title Module1: Imports
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
nltk.download('stopwords')
stop_words=set(stopwords.words("english"))
##############################################################

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [81]:
####
import pandas as pd
import re
A = pd.DataFrame({},columns=['Title','Author','info_abt_paper','text_block'])

def load_row(my_row):
  A.loc[len(A.index)+1] = my_row

with open("/content/drive/MyDrive/CRAN/Unzipped_cran/cran.all.1400", 'r') as x:
  #next(x) # it is used when first line contains column names
  full_string = x.read()
  full_string = full_string.replace('\n',' ').split(".I")[1:]
  for seg in full_string:
    seg = seg.replace(".T","_").replace(".A","_").replace(".B","_").replace(".W","_")
    seg=seg.split("_")[1:]
    load_row(seg[:4])
A.index.name = 'ID'
A=A.reset_index()


In [82]:
# TEXT QUERY

In [83]:
def rem_symbols(line):
  #return re.sub("[^A-Aa-z0-9\s]+",'', line)
  return re.sub('[^A-Za-z0-9\s]+', '', line).lower()

def preprocess_string(x):
  final_list = []
  x = x.replace(',',' ').split(' ')
  for word in x:
        word = rem_symbols(word)
        if word and word not in stop_words:
          word = porter.stem(word)
          final_list.append(word)
  #print("Preprocessed to:- ",final_list)
  return final_list


In [84]:

def rel_text(present1):
  List_present = preprocess_string(present1)
  #List_notpresent = preprocess_string(notpresent1)
  # for now let's consider only one term
  #present = List_present[0]  
  #notpresent = List_notpresent[0]
  with open('/content/drive/MyDrive/CRAN/Dumps_1/Text_combine/merged_Text.txt', 'r') as x:
    next(x)
    count_1 = 0
    rel_doc = [ ]
    for i in x:
      i = i.strip()
      list_doc = i.split(" ")
      term = list_doc[0]   # "Hellow"
      freq = list_doc[1]   # "32"
      docs = list_doc[2:]  # 34_2 43_3 98_1
      if term in List_present:
        # 
        count_1 += 1
        rel_doc+=docs
    if len(rel_doc) == 0:
      # no relevent doc was found
      return None
    else:
      # rel doc found
      rel_list=[]
      for i in rel_doc:
        j,k = list(map(int, i.split("_")))
        rel_list.append((j,k))                    # [(23,45), (34,23), (76,34)] (docId,freq)
      rel_set = set([k for k,l in rel_list])

      desired_list = list(rel_set)    
      # desired_list [23, 34, 12, 120, 43]  list docIds
      filtered_list = []
      for i in set(rel_list):
        if i[0] in desired_list:
          filtered_list.append(i)
      sorted_rel_list = sorted(filtered_list,key=lambda j:j[1],reverse=True)
      return sorted_rel_list
###############################################################

In [85]:
def Display1():
  print("Query:- A intersection B")
  st1 = input("A,B = ").split()
  if len(st1) != 2:
    print("Invalid input")
    return
  l1 = rel_text(st1[0])
  l2 = rel_text(st1[1])
  if l1 is None or l2 is None:
    print("Not Found")
  else:
    l1 = set([i for i,j in l1])
    l2 = set([i for i,j in l2])
    print(list(l1.intersection(l2)))

def Display2():
  print("Query:- A union B")
  st1 = input("A,B = ").split()
  if len(st1) != 2:
    print("Invalid input")
    return
  l1 = rel_text(st1[0])
  l2 = rel_text(st1[1])
  if l1 is None or l2 is None:
    print("Not Found")
  else:
    l1 = set([i for i,j in l1])
    l2 = set([i for i,j in l2])
    print(list(l1.union(l2)))

def Display3():
  print("Query:- A but not B")
  st1 = input("A,B = ").split()
  if len(st1) != 2:
    print("Invalid input")
    return
  l1 = rel_text(st1[0])
  l2 = rel_text(st1[1])
  if l1 is None or l2 is None:
    print("Not Found")
  else:
    l1 = set([i for i,j in l1])
    l2 = set([i for i,j in l2])
    print(list(l1-l2)) 

In [86]:
Display1()  # intersection/conjunction sinusoid wave

Query:- A intersection B
A,B = 
Invalid input


In [87]:
Display2() # union/disjunctions sinusoid wave

Query:- A union B
A,B = 
Invalid input


In [88]:
Display3() # A negation B sinusoidal wave

Query:- A but not B
A,B = 
Invalid input


In [89]:
A[A['ID']==1329].loc[1328,'text_block']  # doc1023 contains term 'sinusoidal' and 'wave'

' some aspects of non-stationary airfoil theory and its practical application .   this paper consists of three notes on the theory of two- dimensional thin airfoils in non-uniform motion .. oscillating airfoil are collected from an earlier paper and are presented in convenient forms for practical application . rigid airfoil passing through a vertical-gust pattern having a sinusoidal distribution of intensity .  the lift is determined as a function of the reduced frequency (which in this case is proportional to the ratio of the airfoil chord and the wave length of the gust pattern) and is presented in the form of a vector diagram . it is shown that the lift acts at the quarter-chord point of the airfoil at all times . calculation of the amplitude of torsional oscillation of a fan blade operating in the wake of a set of pre-rotation vanes .  in a numerical example the amplitude is found to be small even when the vanes are spaced so that the exciting frequency coincides with the natural f

In [90]:
A[A['ID']==2].loc[1,'text_block']  # doc2 contains term wave

" simple shear flow past a flat plate in an incompressible fluid of small viscosity . in the study of high-speed viscous flow past a two-dimensional body it is usually necessary to consider a curved shock wave emitting from the nose or leading edge of the body .  consequently, there exists an inviscid rotational flow region between the shock wave and the boundary layer .  such a situation arises, for instance, in the study of the hypersonic viscous flow past a flat plate .  the situation is somewhat different from prandtl's classical boundary-layer problem . in prandtl's original problem the inviscid free stream outside the boundary layer is irrotational while in a hypersonic boundary-layer problem the inviscid free stream must be considered as rotational .  the possible effects of vorticity have been recently discussed by ferri and libby .  in the present paper, the simple shear flow past a flat plate in a fluid of small viscosity is investigated .  it can be shown that this problem c

In [91]:
A[A['ID']==65].loc[64,'text_block']  # doc65 contains both terms sinusoidal and wave

' convection of a pattern of vorticity through a shock wave .   an arbitrary weak spatial distribution of vorticity can be represented in terms of plane sinusoidal shear waves of all orientations and wave lengths (fourier integral) .  the analysis treats the passage of a single representative weak shear wave through a plane shock and shows refraction and modification of the shear wave with simultaneous generation of an acoustically intense sound wave .  applications to turbulence and to noise in supersonic wind tunnels are indicated . '

# Retrieve Documents Rank based on freq of term in doc

In [92]:
from collections import defaultdict

def Display4():
  print("Query:- ")
  st1 = input("write your query: ")
  st2 = st1.split()
  if len(st2) ==0:
    print("Invalid input")
    return
  freq_dist=  defaultdict(set)
  for i in st2:
    retrieved_tuple=rel_text(i)
    print("retreived:",retrieved_tuple)
    if retrieved_tuple is None:
      pass
    else:
      for j,k in retrieved_tuple:
        if j not in freq_dist:
          # first time
          freq_dist[j] = k
        else:
          # not first time
          freq_dist[j] += k
  # now we have doc and its freq { 23:100, 21:200, 87:30}
  ranked_list =[]
  for i,j in freq_dist.items():
    ranked_list.append((i,j))
  ranked_it = sorted(ranked_list,key=lambda _:_[1],reverse=True)
  print()
  print(ranked_it[:10])
  return 


In [93]:
Display4()

Query:- 
write your query: 
Invalid input


In [94]:
A[A['ID']==64].loc[63,'text_block']  # doc65 contains both terms sinusoidal and wave

' unsteady oblique interaction of a shock wave with plane disturbances .   analysis is made of the flow field produced by oblique impingement of weak plane disturbances of arbitrary profile on a plane normal shock . three types of disturbance are considered .. moves .  the sound wave refracts either as a simple isentropic sound wave or as an attenuating isentropic pressure wave, depending on the angle between the shock and the incident sound wave .  a stationary vorticity wave of constant pressure appears behind the shock . reflects as a sound wave, and a stationary vorticity wave is produced . the shock .  the incident wave refracts as a stationary vorticity wave, and either a sound wave or attenuating pressure wave is also produced .   computations are presented for the first two types of incident wave, over the range of incidence angles, for shock mach numbers of 1, 1.5, and . '

In [95]:
A[A['ID']==1244].loc[1243,'text_block']  # doc65 contains both terms sinusoidal and wave

" on the aerodynamic noise of a turbulent jet .   a new model is advanced for analyzing the broad-spectrum noise of a turbulent jet .  the shear layer bounding the turbulent jet is assumed to play an important role in modifying the / quadrupole sound radiation/ from the interior .  to the sound- emitting small-scale turbulent eddies (with frequencies much higher than those of large-scale eddies), the laminar shear layer has an irregular contour, as if the large-scale turbulent motions were frozen .  the linearized analysis is then applied to the laminar shear layer to relate the acoustic oscillations across it .   the concept of geometrical acoustics is generalized to represent the passage of an acoustic ray through a laminar shear layer . acoustic rays may be traced across the shear layer as transmission and refraction, but they may also be apparently /absorbed/ or /generated/ by the laminar layer .  this /generation/ is visualized as the schematic representation, within the framework

# Next CRAN Queries and Relevance doc as DATAFRAME

In [96]:
import pandas as pd
import re
A = pd.DataFrame({},columns=['query_no','query'])

def load_row(my_row):
  A.loc[len(A.index)+1] = my_row

with open(r"/content/drive/MyDrive/CRAN/Unzipped_cran/cran.qry",'r') as x:
    full_file = x.read()
    full_file = full_file.strip().replace("\n"," ").split(".I")[1:]
    for line in full_file:
        line= line.split(".W")
        a,b = line
        b = b.strip()
        load_row([int(a),b])

B = pd.DataFrame({},columns=['query_no','doc_id','rel_score'])

def load_row_q(my_row):
  B.loc[len(B.index)+1] = my_row

with open(r"/content/drive/MyDrive/CRAN/Unzipped_cran/cranqrel",'r') as x:
    for i in x:
        load_row_q(list(map(int,i.split())))


In [97]:
A.head(2)

,query_no,query
1,1,what similarity laws must be obeyed when const...
2,2,what are the structural and aeroelastic proble...


In [98]:
B.head(2)

,query_no,doc_id,rel_score
1,1,184,2
2,1,29,2


# Zone Relevance score

In [99]:

def rel_zone(present1,zone):
  List_present = preprocess_string(present1)
  #List_notpresent = preprocess_string(notpresent1)
  # for now let's consider only one term
  #present = List_present[0]  
  #notpresent = List_notpresent[0]
  with open(f'/content/drive/MyDrive/CRAN/Dumps_1/{zone}_combine/merged_{zone}.txt', 'r') as x:
    next(x)
    count_1 = 0
    rel_doc = [ ]
    for i in x:
      #print(i)
      i = i.strip()
      list_doc = i.split(" ")
      term = list_doc[0]   # "Hellow"
      freq = list_doc[1]   # "32"
      docs = list_doc[2:]  # 34_2 43_3 98_1
      if term in List_present:
        # 
        count_1 += 1
        rel_doc+=docs
    if len(rel_doc) == 0:
      # no relevent doc was found
      return None
    else:
      # rel doc found
      rel_list=[]
      for i in rel_doc:
        j,k = list(map(int, i.split("_")))
        rel_list.append((j,k))                    # [(23,45), (34,23), (76,34)] (docId,freq)
      rel_set = set([k for k,l in rel_list])

      desired_list = list(rel_set)    
      # desired_list [23, 34, 12, 120, 43]  list docIds
      filtered_list = []
      for i in set(rel_list):
        if i[0] in desired_list:
          filtered_list.append(i)
      sorted_rel_list = sorted(filtered_list,key=lambda j:j[1],reverse=True)
      return sorted_rel_list
###############################################################

In [100]:
from collections import defaultdict

def query(st1,zone):
  #print("Query:- ")
  #st1 = input("write your query: ")
  st2 = st1.split()
  if len(st2) ==0:
    #print("Invalid input")
    return "NONE"
  freq_dist=  defaultdict(set)
  for i in st2:
    retrieved_tuple=rel_zone(i,zone)
    #print("retreived:",retrieved_tuple)
    if retrieved_tuple is None:
      pass
    else:
      for j,k in retrieved_tuple:
        if j not in freq_dist:
          # first time
          freq_dist[j] = k
        else:
          # not first time
          freq_dist[j] += k
  # now we have doc and its freq { 23:100, 21:200, 87:30}
  if freq_dist ==defaultdict(set):
    # empty
    return "NONE"
  ranked_list =[]
  for i,j in freq_dist.items():
    ranked_list.append((i,j))
  ranked_it = sorted(ranked_list,key=lambda _:_[1],reverse=True)
  #print()
  #print(ranked_it[:10])
  return ranked_it


In [101]:
import numpy as np
def val_prec(list1):
  return np.sum(list1)/len(list1)

def cal_precision(true_list,pred_list):
  new_list=[]
  for i in pred_list:
    if i in true_list:
      new_list.append(1)
    else:
      new_list.append(0)
  return np.mean([val_prec(new_list[:i]) for i in range(1,len(new_list)+1)])

def score_query(query_no1,zone):
  # it return true list of relevant documents
  true_list = []
  for i,j in B[B.query_no==query_no1].iloc[:,1:3].iterrows():
    true_list.append((j[0],j[1]))
  t=sorted(true_list,key= lambda _:_[1])

  # it return pred list of relevant documents
  #p=[(51, 26), (874, 22), (486, 17), (792, 14), (1328, 13), (252, 12), (329, 12), (1268, 12), (1147, 11), (1144, 11)]
  p = query(A[A.query_no==query_no1].iloc[0,1],zone)
  """
  print()
  print('Relevent documents:-')
  print(t)
  print("Retrieved documents:-")
  print(p)
  print("MAP:-")
  """
  if p=='NONE':
    #print("found",p)
    p=0
    #####print(p)
    return 0,p,t
  z = cal_precision([i for i,j in t],[i for i,j in p])
  print(z)
  return z,p,t


In [104]:
score_query(2,'Text')
print()

0.0590858944078044



In [ ]:
sni


# Learn weights using Machine Learning

In [105]:
from sklearn.metrics import mean_squared_error

In [106]:
def my_final_retrieved_docs(weights):
  all_queries = list(A.query_no.values)
  value_whole = 0
  #weights_list = [1,2,3,4]
  weights_list = weights
  for que in all_queries:
    #w1,w2,w3,w4 = weights
    # weights = [.1, .2, .3, .4]
    #           'Title' 'Author' 'Text' 'Info'
    the_list = ['Title','Author','Text','Info']
    my_dict = {}
    true_map =[]
    #for i1,w in enumerate(weights):
    for i1,w in enumerate(weights_list):
      MAP, pred_list, true_list = score_query(que,the_list[i1])
      true_vals = [i for i,j in true_list]
      if pred_list ==0 or pred_list is None or pred_list == []:
        pass
      else:
        for val in pred_list:
          #print("entered2_",val[0])
          #print("entered",val)
          x= val[0]
          if x not in my_dict:
            my_dict[x] = 1*w
            true_map += [int(x in true_vals)]
          else:
            my_dict[x] = my_dict[x] + 1*w 
          #print('entered3_',val[0])
        
    # we will have dict {23: .2, 12:. 3, 23:.12}
    # we have list      [ 1       0       1    ]
    #print(pd.DataFrame(my_dict, columns=['doc_id','score']))
    c = pd.DataFrame(columns=['doc_id','score','b'])
    c['doc_id'] = list(my_dict.keys())
    c['score'] = list(my_dict.values())
    c['b'] = true_map
    value_whole +=  mean_squared_error(c['b'],c['score'])
  return value_whole/len(all_queries)


In [107]:
z=my_final_retrieved_docs([.1,.2,.3,.4])
print(z)

0.15435135370562167
0.09337445404849311
0.0
0.07539813029970197
0.0590858944078044
0.0
0.0
0.0
0.0
0.016127027851095342
0.009268526471519167
0.0
0.0
0.0
0.0
0.0
0.003097276516041426
0.0030739165679154667
0.0
0.008221084147427588
0.0032926467418769507
0.0052081448637302154
0.0006608145871168985
0.0
0.0
6.421035021673544e-05
0.0
0.0007882203311117375
0.007043613705785137
0.0
0.0
0.0
0.06055747568913056
0.036594029292943274
0.0
0.0
0.0
0.0
0.0
0.0
0.001461593534580841
0.0022195127641373965
0.0007930147237533535
0.0
0.003358233002218088
0.0009339545787584079
0.0
0.0016902412547685113
0.00201420799536393
0.009909410240778594
0.011425141914221566
0.0
0.0
0.0065696146469682954
0.0
0.01648363453537491
0.018430933146299927
0.0
0.01953891744234797
0.02389344415915563
0.0
0.0
0.0
0.004649298326051649
0.0
0.0
0.0
0.0
0.007254142011510866
0.009232588006882224
0.0
0.03518076397415613
0.02512338278608997
0.0
0.0
0.0
0.0
0.0
0.0
0.002153642784107069
0.006082736007904686
0.0072749083029679746
0.0
0.005

In [108]:
z

0.11485894681301133

In [47]:
0.11741278314988621

## Let's find optimal value of weights using OPTUNA

In [50]:
!pip install optuna

     |████████████████████████████████| 302 kB 35.5 MB/s 
     |████████████████████████████████| 208 kB 72.0 MB/s 
     |████████████████████████████████| 80 kB 7.6 MB/s 
     |████████████████████████████████| 75 kB 3.8 MB/s 
     |████████████████████████████████| 111 kB 50.9 MB/s 
     |████████████████████████████████| 141 kB 44.4 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=0587cc09ab52313b21b03c30ebcf8ef21616518bc8a2ca86cc6421ff2998047c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [51]:
import optuna

In [52]:
def objective(trial):
    w1= trial.suggest_uniform('w1',0,1)
    w2 = trial.suggest_uniform('w2',0,1)
    w3 = trial.suggest_uniform('w3',0,1)
    w4 = trial.suggest_uniform('w4',0,1)
    return my_final_retrieved_docs([w1,w2,w3,w4])
    

In [53]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=50)

[I 2021-09-12 07:37:47,078] A new study created in memory with name: no-name-a81f82cb-9f3e-43de-a5f1-c357a351cfb5
[I 2021-09-12 07:39:22,700] Trial 0 finished with value: 0.4655026219927819 and parameters: {'w1': 0.2735769219458293, 'w2': 0.3199624888274558, 'w3': 0.5806686701276269, 'w4': 0.006714254503807027}. Best is trial 0 with value: 0.4655026219927819.
[I 2021-09-12 07:40:59,394] Trial 1 finished with value: 0.14346286941166916 and parameters: {'w1': 0.23536528117779254, 'w2': 0.8548543490341234, 'w3': 0.26512361442811927, 'w4': 0.8934128233543567}. Best is trial 1 with value: 0.14346286941166916.
[I 2021-09-12 07:42:34,942] Trial 2 finished with value: 0.12793901011137213 and parameters: {'w1': 0.343038224811524, 'w2': 0.8898865890590072, 'w3': 0.18713554791403908, 'w4': 0.7378138598347496}. Best is trial 2 with value: 0.12793901011137213.
[I 2021-09-12 07:44:09,427] Trial 3 finished with value: 0.05326098576867339 and parameters: {'w1': 0.23950166428794306, 'w2': 0.46146761696

In [54]:
v=study.best_params
v

{'w1': 0.009946558660320098,
 'w2': 0.267004428647879,
 'w3': 0.00032022522138083764,
 'w4': 0.30108544888047406}

In [54]:
{'w1': 0.009946558660320098,
 'w2': 0.267004428647879,
 'w3': 0.00032022522138083764,
 'w4': 0.30108544888047406}

In [56]:
z=my_final_retrieved_docs([v['w1'],v['w2'],v['w3'],v['w4']])
print(z)

0.004989126915944071


# ['Title', 'Author', 'Text', 'Info'] ===>> 
#weights [0.6296626602841014, 0.8168310496870982, 0.9606763876973095, 0.2516580343220661]

we see maximum weight is given in to 'Text' zone and minimum weight to 'Info' zone WHICH  MAKES SENSE as Text zone is most important for retrieval